In [17]:
from pyschedule import Scenario, solvers, plotters, alt

### Site Configuartion

Scanners

In [18]:
MRI_Scanner = 0
PET_Scanner = 1

Uptake Rooms

In [19]:
UptakeRooms_Count = 2

Hot Toilet

In [20]:
HotToilet_Count = 1

Workforce

In [21]:
Radiographers = 0
Technologists = 2
Clinical_Assistants = 2
Admin = 1

Operating Hours

In [22]:
Site_Operating_Hours = 12
Site_Operating_Hours_FlexHours = 2
Technologist1_Operating_Hours = 12
Technologist2_Operating_Hours = 12
Clinical_Asst1_Operating_Hours = 12
Clinical_Asst2_Operating_Hours = 12
Admin_Operating_Hours = 12

Scan Limits

In [23]:
Technologist1_Scan_Limit = 10
Technologist2_Scan_Limit = 10

Estimated Scan Demand

In [24]:
PET30_Scan_Count = 15
PET60_Scan_Count = 4

### Create Scenario

In [25]:
# Load pyschedule and create a scenario with ten steps planning horizon
S = Scenario('PET_Schedule',horizon=720)

### Create Resources

In [26]:
# Scanner resources
PET_Scanner = S.Resource('PET_Scanner',num=1)
MRI_Scanner = S.Resource('MRI_Scanner',num=0)

In [27]:
UptakeRoom = S.Resources('UptakeRoom',num=2)

In [28]:
# Workforce resources
Clinical_Assistant = S.Resource('Clinical_Assistant',num=1)
Technologist = S.Resource('Technologist',num=2)
Radiographer = S.Resource('Radiographer',num=0)
Admin = S.Resource('Admin',num=1)

Tasks

In [29]:
# Create three tasks with lengths 1,2 and 3
Doc_1 = S.Task('Doc_1',length=15)
SQues_1 = S.Task('SQues_1',length=10)
BVitals_1 = S.Task('BVitals_1',length=2)
Cannulate_1 = S.Task('Cannulate_1',length=5) 
SQuesValidate_1 = S.Task('SQuesValidate_1',length=2)
Inject_1 = S.Task('Inject_1',length=4)
PRelax_1 = S.Task('PRelax_1',length=60)
Scan30_1 = S.Task('Scan30_1',length=30,delay_cost=1)

Doc_2 = S.Task('Doc_2',length=15)
SQues_2 = S.Task('SQues_2',length=10)
BVitals_2 = S.Task('BVitals_2',length=2)
Cannulate_2 = S.Task('Cannulate_2',length=5) 
SQuesValidate_2 = S.Task('SQuesValidate_2',length=2)
Inject_2 = S.Task('Inject_2',length=4)
PRelax_2 = S.Task('PRelax_2',length=60)
Scan60_2 = S.Task('Scan60_2',length=60,delay_cost=1)

Who Can Do What Tasks

In [30]:
Doc_1 += Clinical_Assistant | Admin | Technologist
BVitals_1 += Clinical_Assistant | Technologist 
SQues_1 += Clinical_Assistant | Technologist
SQuesValidate_1 += UptakeRoom[0] | UptakeRoom[1], Clinical_Assistant | Technologist
Cannulate_1 += UptakeRoom[0] | UptakeRoom[1], Clinical_Assistant | Technologist
PRelax_1 += UptakeRoom[0] | UptakeRoom[1]
Inject_1 += UptakeRoom[0] | UptakeRoom[1] , Technologist
Scan30_1 += PET_Scanner, Technologist

Doc_2 += Clinical_Assistant | Admin | Technologist
BVitals_2 += Clinical_Assistant | Technologist 
SQues_2 += Clinical_Assistant | Technologist
SQuesValidate_2 += UptakeRoom[0] | UptakeRoom[1] , Clinical_Assistant | Technologist
Cannulate_2 += UptakeRoom[0] | UptakeRoom[1], Clinical_Assistant | Technologist
PRelax_2 += UptakeRoom[0] | UptakeRoom[1]
Inject_2 += UptakeRoom[0] | UptakeRoom[1], Technologist
Scan60_2 += PET_Scanner, Technologist

Task Precedence

In [31]:
S+= Doc_1 < BVitals_1
S+= BVitals_1 < SQues_1
S+= SQues_1 < SQuesValidate_1
S+= SQuesValidate_1 < Cannulate_1
S+= Cannulate_1 < Inject_1
S+= Inject_1 < PRelax_1
S+= PRelax_1 < Scan30_1
S+= Doc_1 + 150 > Scan30_1
S+= Cannulate_1 + 70 < Scan30_1


S+= Doc_2 < BVitals_2
S+= BVitals_2 < SQues_2
S+= SQues_2 < SQuesValidate_2
S+= SQuesValidate_2 < Cannulate_2
S+= Cannulate_2 < Inject_2
S+= Inject_2 < PRelax_2
S+= PRelax_2 < Scan60_2
S+= Doc_2 + 180 > Scan60_2
S+= Cannulate_2 + 70 < Scan60_2

In [32]:
print(S)

###############################################

SCENARIO: PET_Schedule / horizon: 720

OBJECTIVE: Scan30_1+Scan60_2

RESOURCES:
PET_Scanner
MRI_Scanner
UptakeRoom0
UptakeRoom1
Clinical_Assistant
Technologist
Radiographer
Admin

TASKS:
Doc_1 : Clinical_Assistant|Admin|Technologist
SQues_1 : Clinical_Assistant|Technologist
BVitals_1 : Clinical_Assistant|Technologist
Cannulate_1 : UptakeRoom0|UptakeRoom1,Clinical_Assistant|Technologist
SQuesValidate_1 : UptakeRoom0|UptakeRoom1,Clinical_Assistant|Technologist
Inject_1 : UptakeRoom0|UptakeRoom1,Technologist
PRelax_1 : UptakeRoom0|UptakeRoom1
Scan30_1 : PET_Scanner,Technologist
Doc_2 : Clinical_Assistant|Admin|Technologist
SQues_2 : Clinical_Assistant|Technologist
BVitals_2 : Clinical_Assistant|Technologist
Cannulate_2 : UptakeRoom0|UptakeRoom1,Clinical_Assistant|Technologist
SQuesValidate_2 : UptakeRoom0|UptakeRoom1,Clinical_Assistant|Technologist
Inject_2 : UptakeRoom0|UptakeRoom1,Technologist
PRelax_2 : UptakeRoom0|UptakeRoom1
Scan60_2 :

Resource Requirement

In [33]:
# compute and print a schedule
solvers.mip.solve(S,msg=1)
print(S.solution())

INFO: execution time for solving mip (sec) = 144.06198406219482
INFO: objective = 238.0
[(Doc_1, Clinical_Assistant, 0, 15), (BVitals_1, Clinical_Assistant, 15, 17), (Doc_2, Admin, 16, 31), (SQues_1, Clinical_Assistant, 17, 27), (SQuesValidate_1, Clinical_Assistant, 27, 29), (SQuesValidate_1, UptakeRoom0, 27, 29), (Cannulate_1, Technologist, 29, 34), (Cannulate_1, UptakeRoom1, 29, 34), (BVitals_2, Clinical_Assistant, 33, 35), (Inject_1, Technologist, 35, 39), (Inject_1, UptakeRoom1, 35, 39), (SQues_2, Clinical_Assistant, 35, 45), (PRelax_1, UptakeRoom0, 39, 99), (SQuesValidate_2, Technologist, 45, 47), (SQuesValidate_2, UptakeRoom1, 45, 47), (Cannulate_2, Clinical_Assistant, 48, 53), (Cannulate_2, UptakeRoom1, 48, 53), (Inject_2, Technologist, 55, 59), (Inject_2, UptakeRoom1, 55, 59), (PRelax_2, UptakeRoom1, 71, 131), (Scan30_1, PET_Scanner, 104, 134), (Scan30_1, Technologist, 104, 134), (Scan60_2, PET_Scanner, 134, 194), (Scan60_2, Technologist, 134, 194)]


In [34]:
plotters.matplotlib.plot(S,img_filename='op3.png')